# Task II. Presets 

In [6]:
import random
import spacy
from spacy.training import Example
import json
from spacy.pipeline.textcat_multilabel import DEFAULT_MULTI_TEXTCAT_MODEL

In [7]:
with open("taxiandcars.json") as f:
    data = json.load(f)

In [8]:
nlp = spacy.load("en_core_web_md")

# Task II. Part I

## Problem statement

Застосувати компонент TextCategorizer для визначення намірів.

## Solution

In [9]:
config = {
    "threshold": 0.5,
    "model": DEFAULT_MULTI_TEXTCAT_MODEL,
}

textcat = nlp.add_pipe("textcat_multilabel", config=config)

In [10]:
textcat.add_label("GetRide")
textcat.add_label("GetCarsAvailable")
textcat.add_label("ReverseCar")

1

# Task II. Part II

# Problem statement

Дані для навчання за тематикою варіанту обрати самостійно або скористатись вказаним файлом (utterance містить висловлювання, intent - намір).

# Solution

In [11]:
train_1 = []
train_2 = []
train_3 = []
for dialogue in data:
    for turn in dialogue["turns"]:
        text = turn["utterance"]
        intent = None
        for action in turn["frames"][0]["actions"]:
            if action["act"] == "INFORM_INTENT":
                intent = action["canonical_values"][0]
                break
        if intent == "GetRide":
            train_1.append((text, {"cats": {intent: True}}))
        if intent == "GetCarsAvailable":
            train_2.append((text, {"cats": {intent: True}}))
        if intent == "ReserveCar":
            train_3.append((text, {"cats": {intent: True}}))

In [21]:
train_examples_1 = [Example.from_dict(nlp.make_doc(text), label) for text, label in train_1]
train_examples_2 = [Example.from_dict(nlp.make_doc(text), label) for text, label in train_2]
textcat.initialize(lambda: train_examples_1, nlp=nlp)
textcat.initialize(lambda: train_examples_2, nlp=nlp)

# Task II. Part III

## Problem statement

Навчити компонент.

# Solution

In [22]:
epochs = 100
with nlp.select_pipes(enable="textcat"):
    optimizer = nlp.resume_training()
    for i in range(epochs):
        for example in train_examples_1:
            nlp.update([example], sgd=optimizer)

# Task II. Part IV

## Problem statement

Продемонструвати роботу.

## Solution

In [23]:
text = "I'll pick it up around 10:30 in the morning, keeping it until next Thursday"
doc = nlp(text)
intent_scores = doc.cats
predicted_intent = max(intent_scores, key=intent_scores.get)

In [24]:
print("Text:", text)
print("Predicted Intent:", predicted_intent)
print("Intent Scores:", intent_scores)

Text: I'll pick it up around 10:30 in the morning, keeping it until next Thursday
Predicted Intent: GetCarsAvailable
Intent Scores: {'GetRide': 0.6222334504127502, 'GetCarsAvailable': 0.8771541714668274, 'ReverseCar': 0.7279171943664551}


In [39]:
text = "Is a shared ride okay?"
doc = nlp(text)
intent_scores = doc.cats
predicted_intent = max(intent_scores, key=intent_scores.get)

In [40]:
print("Text:", text)
print("Predicted Intent:", predicted_intent)
print("Intent Scores:", intent_scores)

Text: Is a shared ride okay?
Predicted Intent: ReverseCar
Intent Scores: {'GetRide': 0.03920861706137657, 'GetCarsAvailable': 0.07123273611068726, 'ReverseCar': 0.26356327533721924}


In [27]:
text = "Reserve a car on Monday 5:00 am."
doc = nlp(text)
intent_scores = doc.cats
predicted_intent = max(intent_scores, key=intent_scores.get)

In [28]:
print("Text:", text)
print("Predicted Intent:", predicted_intent)
print("Intent Scores:", intent_scores)

Text: Reserve a car on Monday 5:00 am.
Predicted Intent: ReverseCar
Intent Scores: {'GetRide': 0.0997655913233757, 'GetCarsAvailable': 0.07094454020261765, 'ReverseCar': 0.8834327459335327}
